# data cleaning

turn tweet data set into workable dataframe using pyspark

In [174]:
from pyspark import SparkContext

In [175]:
sc = SparkContext.getOrCreate()

In [176]:
tweets = sc.textFile("file:///Users/tijsvandenheuvel/bin/tweets/dev_r.txt")

In [177]:
tweets_clean = tweets.flatMap(lambda x: x.split(' ')).filter(lambda x:x!='O').filter(lambda x:x!='').filter(lambda x:x!='<STOP>')


In [178]:
tweets_string = tweets_clean.reduce(lambda x,y:x+' '+y)
import re
res = re.compile('\<START\>').split(tweets_string)
res = list(filter(None, res))
tweet_rdd = sc.parallelize(res)

In [179]:
tweets = tweet_rdd.zipWithIndex().map(lambda x: (x[1],x[0]))

In [180]:
# get emojis out of text
emoji_rdd = tweets.map(lambda x: (x[0],re.findall(":.*?:",x[1]))).map(lambda x:(x[0],' '.join(x[1])))
text_rdd = tweets.map(lambda x: (x[0],re.sub(":.*?:","",x[1])))

combined = text_rdd.leftOuterJoin(emoji_rdd).map(lambda x:(x[0],x[1][0],x[1][1]))

In [181]:
combined.collect()

[(0,
  ' No object is so beautiful that under certain conditions it will not look ugly Oscar Wilde ↺ RT  … ',
  ':red_heart:'),
 (1,
  ' Cant expect different results doing the same thingdoing stuff different from now on  🏻 \u200d  ️ ',
  ':person_shrugging: :female_sign:'),
 (2,
  ' “ Lets go Marcus ” “ Shiiit where we goin Home ” Marcus Peters  ',
  ':face_with_tears_of_joy:'),
 (3,
  ' Asahd really is a grown man in the body of a 1 year old  ',
  ':face_with_tears_of_joy:'),
 (4,
  ' Yoongi Tweet Hello Im Min fell on Butt What the  Min ',
  ':face_with_tears_of_joy:'),
 (5,
  ' we cannot afford İSJK  n AQ play havoc with our lives in Kashmir Yes these are the independent Kash … ',
  ':backhand_index_pointing_down:'),
 (6,
  ' ranks 6th in January Idol Group Brand Reputation  1Keep using 2Search GOT 7 on Naver  htt … ',
  ':party_popper: :backhand_index_pointing_down:'),
 (7,
  ' Ok people are really trying to kill themselves with this Tide Pod challenge Who tf and why tf  🏽 \u200d  

In [188]:
from pyspark.sql import SQLContext

s   = SQLContext(sc)
df = s.createDataFrame(combined, ['id', 'text','emoji'])


df.repartition(1).write.csv('file:///Users/tijsvandenheuvel/bin/tweets/outv2', sep=',', header=True)
